# Einstieg Python

## Import der benötigten Module:

In [1]:
import geopandas as gp
import matplotlib.pyplot as plt
import warnings

Warnungen können für's Erste unterdrückt werden

In [ ]:
warnings.filterwarnings('ignore')

### Nun laden Sie die Daten, die Bezirksdaten (Geopackage aus dem Seminar) und die Bars (von OpenStreetMap extrahiert) ein

In [ ]:
districts = gp.read_file("./data/bezirke_EPSG32633.shp")
bars = gp.read_file("./data/bars_bln.gpkg")

Schauen Sie sich den Dataframe des *Bezirke-Layers* einmal an:

In [ ]:
print('Das ist der Bezirke Dataframe.')
print(districts)

Ebenso die im Geopackage enthaltenen Informationen der Bars, also den *Bars-Layer*

In [ ]:
print('Das ist der Bar Dataframe.')
print(bars)

...ganz schön viele Informationen, die da von *OpenStreetMap* kommen. Finden Sie heraus, wie viele Zeilen und Spalten der *Dataframe* hat.

Um einen besseren Überblick zu bekommen, welche Spalten im *Dataframe* sind (ähnlich zu den Spalten von *Attributtabellen* in *QGIS*), *printen* Sie sich lediglich die Spalten.

In [ ]:
print('Das sind alle Spalten des Bar-Dataframes.')
print(bars.columns.tolist())

Schauen Sie sich die Spaltennamen an.
Wie würdet Sie die Datenqualität einschätzen? Wie ist die Datenqualität insgesamt von *OpenStreetMap* ein zu schätzen bzw. von anderen Crowd Sourced Datenwebsiten wie *Wikipedia* oder *WikiMaps* ? Diskutieren Sie mit Ihrem_r Nachbar_in.

### Spatial Join

Nun wollen wir herausfinden, wie viele Bars es pro Bezirk gibt. Dazu führen Sie einen *Spatial Join* aus. Eine visuelle und mathematische Erklärung der verschiedenen Arten von *Spatial Joins* finden Sie [hier](https://en.wikipedia.org/wiki/DE-9IM).
In *QGIS* vollzieht man folgenden *Spatial Join* über das Tool *Join Attributes by Location* aus der *Processing Toolbox*.
Schauen Sie sich an, was die unterschiedlichen Predicates bedeuten (intersects, contains, equals, touches, overlaps, within, crosses).

In [ ]:
districts.sjoin(bars, predicate = 'contains')

Schauen Sie sich dasm Ergebnis genau an. Ist es das was sie erwartet haben? Diskutieren Sie mit Ihre_m Nachbar_in.

<details>

<summary>Antwort - klicken zum ausklappen</summary>
Geographische Datensätze haben meißt ein <i>Coordinate Reference System (CRS)</i>. Ohne ein <i>CRS</i> können räumliche Daten nicht im Raum verortet werden. Da die Daten aus unterschiedlichen Datensquellen kommen (FIS-Broker, dem Datenportal der Stadt Berlin, und OpenStreetMap), liegt es nahe, dass die <i>CRS</i> unterschiedlich sind. Hinweis: Nur weil Daten aus derselben Datenquelle kommen, heißt das nicht zwangsläufig, dass Sie das gleiche <i>CRS</i> haben. Deswegen ist das Ergebnis was Sie bekommen, ein Dataframe mit 0 Reihen, also ein leerer Dataframe. Die Daten liegen nicht "übereinander", und deswegen liegen auch keine Bars in Bezirken. Schauen wir uns einmal die Koordinatensysteme an, in denen die Daten vorliegen.

</details>

In [ ]:
print('Das ist das CRS vom Bezirke Layer')
print(districts.crs)

In [ ]:
print('Das ist das CRS vom Bars Layer')
print(bars.crs)

Die CRS sind unterschiedlich. Das sehen Sie auch deutlich, wenn Sie sich die Definition der `geometry`-Spalte angucken.

In [ ]:
print('Geometry-Spalte von Bars')
print(bars.geometry)

In [ ]:
print('Geometry-Spalte von Bezirke')
print(districts.geometry) 

Sehen Sie, wie unterschiedlich die Zahlen sind? *QGIS* verwaltet die Koordinatensysteme für uns *on-the-fly*.
Auch, wenn die Inputdaten unterschiedliche *CRS* haben, transformiert *QGIS* die Inputdaten für uns *on-the-fly* in das *CRS*, welches in der *QGIS-Datei* für das Projekt festgelegt ist.

*Geopandas* (und vermutlich jede andere Bibliothek in Python, R,...) vergibt uns unterschiedliche *CRS* nicht so leicht. Hier müssen wir die *CRS* manuell verwalten. Damit Sie herausfinden können, welche Punkte (Bars) in welchem Polygon (Bezirke) liegen, müssen Sie die Dateien in das selbe *CRS* bringen.

Dazu müssen Sie einen der beiden Dataframes reprojizieren. Hinweis: https://spatialreference.org/ ist eine Datenbank, in der alle gängigen Projektionen abgespeichert sind. Geben Sie dort den EPSG Code (EPSG:32633) des Bezirke Layers ein, erhalten Sie Informationen rund um dieses Koordinatensystem. Es gibt auch noch Websiten wie http://mapref.org/. Unter http://mapref.org/CollectionofCRSinEurope.html werden beispielsweise alle Europäischen CRS gesammelt.

In [ ]:
bars = bars.to_crs(districts.crs)

Nun haben Sie die Daten reprojiziert und Sie können den Spatial Join ausführen. *Geopandas* hat noch eine weitere Methode, `.set_crs()` (https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_crs.html). Versuchen Sie anhand der Dokumentation den Unterschied zu der Methode `to_crs()` sich zu erklären (https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html?highlight=to_crs)

<details>

<summary>Antwort - klicken zum ausklappen</summary>
<code>.to_crs()</code> <b>transformiert</b> die geographischen Daten in ein <b>neues</b> <i>CRS</i>. Die ist eine mathemathische Operation und wird durch eine wohldefinierte Transformationsfunktion vorgenommen. <code>.set_crs()</code> ändert dagegen lediglich die Definition des <i>CRS</i> in ein anderes, ohne die Koordinaten zu transformieren. Die Definition eines <i>CRS</i> kann beispielsweise folgenderweise aussehen (EPSG: 25833): <code>+proj=utm +zone=33 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs</code> Ich könnte diesen Text einfach durch einen anderen ersetzen (EPSG: 25832): <code> +proj=utm +zone=32 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs</code>. Für den Computer liegen die Koordinaten nun in <code>25832</code> vor, sie wurden jedoch nicht transformiert. Somit wäre unsere Datengrundlage fehlerhaft, die Daten würden nicht an ihrem eigentlichen Ort auf dem Geoid/Ellipsoid/... verortet werden.

</details>

In [ ]:
barsInDistrict = districts.sjoin(bars, predicate = 'contains')

Wollen Sie zählen, wie viele Bars es pro Bezirk gibt, müssen Sie folgende Zeile Code ausführen

In [ ]:
barsInDistrict.groupby('BEZIRK_NAM').size().rename('count')

Das war eine kleine Einführung in Programmierung mit Python. Wenn Sie Lust haben, tiefer in Python und geographische Methoden in Python ein zu steigen, kommen Sie in den Kurs Forgeschrittene Methoden der Geoinformationsverarbeitung nächstes Semester. Vielen Dank!